In [1]:
import pandas as pd
import numpy as np
import time
from tqdm import tqdm

train_data = pd.read_csv('train_update.csv')
test_data = pd.read_csv('test_update.csv')

train_data = train_data.drop('origin',axis = 1)
test_data = test_data.drop(['item_1','item_2','item_3','item_4','item_5','item_6',
                          'item_7','item_8','item_9','item_10','item_11','item_12',
                          'item_13','item_14','item_15','item_16','item_17','item_18',
                          'item_19','item_20','item_21','item_22','item_23','item_24','origin'],axis =1)
train_data['item_22'] = train_data['item_22'].replace(np.nan, 0)
train_data['item_23'] = train_data['item_23'].replace(np.nan, 0)

train_data = train_data.sort_values(by=['user_id','date'],ascending=[True, False])
train_data = train_data.reset_index(drop = True)

train_data_test1 = train_data[['user_id','date','item_1','item_2','item_3',
                              'item_4','item_5','item_6','item_7','item_8',
                              'item_9','item_10','item_11','item_12','item_13',
                              'item_14','item_15','item_16','item_17','item_18',
                              'item_19','item_20','item_21','item_22','item_23','item_24']]

user_id_list = list(set(train_data_test1['user_id']))
user_id_list.sort()

col_name = list()
for num1 in range(1,25):
    col_name.append('item'+str(num1)+'_buy')

#对产品列进行处理，用户第一次出现的那行产品信息不变，后续日期行进行差分，差分结果为1表示用户在这个日期新购买了该产品，
#-1表示用户在这个日期卖出了该产品，0表示用户未对产品进行操作，将差分结果作为训练的目标标签
train_data_all = pd.DataFrame(columns = col_name)
for user_id_num in tqdm(user_id_list):
    train_data_one = train_data_test1[train_data_test1.user_id == user_id_num]
    train_data_one_temp = train_data_one.drop(['user_id','date'],axis=1)
    train_data_one_matrix = train_data_one_temp.values
    row_num = len(train_data_one_matrix)
    if row_num == 1:
        train_data_one_matrix = train_data_one_matrix
    else:
        for i in range(row_num-1): 
            train_data_one_matrix[i] = train_data_one_matrix[i+1]*(-1)+train_data_one_matrix[i]
    train_data_one_new = pd.DataFrame(train_data_one_matrix,columns=col_name)
    train_data_all = pd.concat([train_data_all, train_data_one_new],ignore_index = True)

100%|████████████████████████████████████████████████████████████████████████████| 39697/39697 [28:50<00:00, 22.94it/s]


In [2]:
train_data1 = pd.concat([train_data,train_data_all] ,axis =1)

train_data1 = train_data1.drop(['item_1','item_2','item_3',
                              'item_4','item_5','item_6','item_7','item_8',
                              'item_9','item_10','item_11','item_12','item_13',
                              'item_14','item_15','item_16','item_17','item_18',
                              'item_19','item_20','item_21','item_22','item_23','item_24'],axis =1)

train_data1.to_csv('train_data1.csv',index = 0)

In [3]:
#去除日期标签，新增月标记，因为用户的产品购买存在一定的月份特征
train_data1['date']=pd.to_datetime(train_data1['date'],format='%Y-%m-%d')
train_data1['month_label'] = train_data1['date'].map(lambda x: x.month)

for num in col_name:
    train_data1[num] = train_data1[num].map(lambda x: 0 if x<0 else x)

train_data1['use_time'] = train_data1['use_time'].map(lambda x: 0 if x<0 else x)
train_data1 = train_data1.drop('date',axis =1)
test_data = test_data.drop('date',axis = 1)
test_data['month_label'] = 5

train_data_x = train_data1.drop(['item1_buy','item2_buy','item3_buy','item4_buy',
                             'item5_buy','item6_buy','item7_buy','item8_buy','item9_buy','item10_buy',
                             'item11_buy','item12_buy','item13_buy','item14_buy','item15_buy','item16_buy',
                             'item17_buy','item18_buy','item19_buy','item20_buy','item21_buy','item22_buy',
                             'item23_buy','item24_buy'],axis = 1)
train_data_y = train_data1[['item1_buy','item2_buy','item3_buy','item4_buy',
                             'item5_buy','item6_buy','item7_buy','item8_buy','item9_buy','item10_buy',
                             'item11_buy','item12_buy','item13_buy','item14_buy','item15_buy','item16_buy',
                             'item17_buy','item18_buy','item19_buy','item20_buy','item21_buy','item22_buy',
                             'item23_buy','item24_buy']]

train_data_x['origin'] = 'train'
test_data['origin'] = 'test'
matrix = pd.concat([train_data_x,test_data],ignore_index = True,sort = False)
print(matrix.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561786 entries, 0 to 561785
Data columns (total 19 columns):
user_id                   561786 non-null int64
user_country              561786 non-null object
sex                       561786 non-null object
age                       561786 non-null int64
new_user_type             561786 non-null float64
user_seniority            561786 non-null int64
user_type                 561786 non-null float64
user_type_at_begin        561786 non-null float64
user_relation_at_begin    561786 non-null object
live_country_type         561786 non-null object
born_country_type         561786 non-null object
user_join_channel         561786 non-null object
province_code             561786 non-null float64
activity_type             561786 non-null float64
user_segmentation         561786 non-null object
use_time                  561786 non-null int64
user_income               561786 non-null int64
month_label               561786 non-null int64
origin  

In [4]:
#对特征列进行Labelencoder
from sklearn.preprocessing import LabelEncoder

lbe_user_id = LabelEncoder()
matrix['user_id_lbe'] = lbe_user_id.fit_transform(matrix['user_id'])

cate_col = ['user_country','sex','new_user_type','user_type','user_type_at_begin','user_relation_at_begin',
           'live_country_type','born_country_type','user_join_channel','province_code','activity_type','user_segmentation',
           'month_label']

for name in cate_col:
    lbe_temp = LabelEncoder()
    matrix[name] = lbe_temp.fit_transform(matrix[name])

label_name =['user_country','sex','age','new_user_type','user_seniority',
            'user_type','user_type_at_begin','user_relation_at_begin','live_country_type',
            'born_country_type','user_join_channel','province_code','activity_type',
            'user_segmentation','use_time','month_label']

#对特征列进行类型变换以减少内存消耗
for name in label_name:
    matrix[name] = matrix[name].astype('int16')
    
matrix['user_income'] = matrix['user_income'].astype('int32')
matrix['user_id_lbe'] = matrix['user_id_lbe'].astype('int32')

train_data_x = matrix[matrix.origin == 'train']
test_data = matrix[matrix.origin == 'test']

print(train_data_x.describe())

train_data_x.to_csv('train_data_x.csv',index = 0)
train_data_y.to_csv('train_data_y.csv',index = 0)
test_data.to_csv('test_data.csv',index = 0)


            user_id   user_country            sex            age  \
count  5.326240e+05  532624.000000  532624.000000  532624.000000   
mean   8.248920e+05      15.018585       0.546990      40.372525   
std    4.330362e+05       0.867818       0.497788      17.174854   
min    1.607400e+04       0.000000       0.000000       2.000000   
25%    4.397070e+05      15.000000       0.000000      24.000000   
50%    9.212120e+05      15.000000       1.000000      39.000000   
75%    1.194005e+06      15.000000       1.000000      51.000000   
max    1.548191e+06      42.000000       1.000000     114.000000   

       new_user_type  user_seniority      user_type  user_type_at_begin  \
count  532624.000000   532624.000000  532624.000000       532624.000000   
mean        0.061826       80.568529       0.001744            0.991572   
std         0.240839       66.771352       0.041727            0.108040   
min         0.000000        0.000000       0.000000            0.000000   
25%         

In [35]:
train_data_x = pd.read_csv('train_data_x.csv')
train_data_y = pd.read_csv('train_data_y.csv')
test_data = pd.read_csv('test_data.csv')

train_data_x = train_data_x.drop(['origin','user_id'],axis = 1)
all_data = pd.concat([train_data_x,train_data_y],axis = 1)

test_data = test_data.drop('origin',axis = 1)
test_data1 = test_data.drop('user_id',axis = 1)

#all_data.to_csv('all_data.csv',index=0)
#将24个目标标签列整合成1列，如果用户购买了某一产品，则在标签列显示相应产品序号
all_data1 = all_data.set_index(['user_country', 'sex', 'age', 'new_user_type', 'user_seniority', 'user_type', 
                                'user_type_at_begin', 'user_relation_at_begin', 'live_country_type', 
                                'born_country_type', 'user_join_channel', 'province_code', 'activity_type', 
                                'user_segmentation', 'use_time', 'user_income', 'month_label', 'user_id_lbe' ])
all_data1 = all_data1.stack().reset_index()
#print(all_data1)

all_data1['level_18'] = all_data1['level_18'].map({'item1_buy': 0, 'item2_buy': 1,'item3_buy': 2,
                                                  'item4_buy': 3, 'item5_buy': 4,'item6_buy': 5,
                                                  'item7_buy': 6, 'item8_buy': 7,'item9_buy': 8,
                                                  'item10_buy': 9, 'item11_buy': 10,'item12_buy': 11,
                                                  'item13_buy': 12, 'item14_buy': 13,'item15_buy': 14,
                                                  'item16_buy': 15, 'item17_buy': 16,'item18_buy': 17,
                                                  'item19_buy': 18, 'item20_buy': 19,'item21_buy': 20,
                                                  'item22_buy': 21, 'item23_buy': 22,'item24_buy': 23})

#print(all_data1)

all_data1['buy'] = all_data1['level_18'].mul(all_data1[0])
#print(all_data1[all_data1.buy != 0])

all_data2 = all_data1[all_data1.buy != 0]

all_data2 = all_data2.drop(['level_18',0],axis =1)
#print(all_data2)

all_data2['buy'] = all_data2['buy'].astype('int32')

#print(all_data2['buy'].value_counts())

test_data2 = test_data1.values
#print(test_data2)

all_data3 = all_data2.drop(['buy'],axis=1).values

all_data4 = all_data2['buy'].values
#print(all_data4)

#进行Xgboost的预测
from sklearn.model_selection import train_test_split
import xgboost as xgb

X = all_data3
Y = all_data4

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
#加载numpy的数组到DMatrix对象
xg_train = xgb.DMatrix(X_train, label=y_train)
xg_test = xgb.DMatrix( X_test, label=y_test)
#1.训练模型
# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softprob'
param['eval_metric'] = 'mlogloss'
# scale weight of positive examples
param['eta'] = 0.01
param['max_depth'] = 15
#param['silent'] = 0
#param['nthread'] = 4
param['num_class'] = 24
param['lamdba'] = 2
#param['colsample_bytree'] = 0.8
#param['subsample'] = 0.9
 
watchlist = [ (xg_train,'train'), (xg_test, 'test') ]
num_round = 10000
bst = xgb.train(param, xg_train, num_round, watchlist, early_stopping_rounds=100, verbose_eval=25 );

test_data_input = xgb.DMatrix(test_data2)
pred = bst.predict( test_data_input );
print (pred)


[0]	train-mlogloss:3.13889	test-mlogloss:3.14266
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 100 rounds.
[25]	train-mlogloss:2.5736	test-mlogloss:2.66398
[50]	train-mlogloss:2.26507	test-mlogloss:2.42824
[75]	train-mlogloss:2.05085	test-mlogloss:2.27489
[100]	train-mlogloss:1.88865	test-mlogloss:2.16662
[125]	train-mlogloss:1.76125	test-mlogloss:2.08751
[150]	train-mlogloss:1.65864	test-mlogloss:2.02799
[175]	train-mlogloss:1.57526	test-mlogloss:1.9823
[200]	train-mlogloss:1.50543	test-mlogloss:1.94756
[225]	train-mlogloss:1.44614	test-mlogloss:1.921
[250]	train-mlogloss:1.39647	test-mlogloss:1.89976
[275]	train-mlogloss:1.35345	test-mlogloss:1.88319
[300]	train-mlogloss:1.31622	test-mlogloss:1.871
[325]	train-mlogloss:1.28422	test-mlogloss:1.86218
[350]	train-mlogloss:1.25602	test-mlogloss:1.85595
[375]	train-mlogloss:1.23034	test-mlogloss:1.85184
[400]	train-mlogloss:1.20757	test-mlogloss:

In [13]:
submit = pd.DataFrame(pred,columns = ['ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1',
                        'ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1',
                        'ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1',
                        'ind_deco_fin_ult1','ind_deme_fin_ult1','ind_dela_fin_ult1',
                        'ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1',
                        'ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1',
                        'ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1',
                        'ind_nomina_ult1','ind_nom_pens_ult1','ind_recibo_ult1'])

In [14]:
print(submit)

       ind_ahor_fin_ult1  ind_aval_fin_ult1  ind_cco_fin_ult1  \
0               0.001708           0.001740          0.115687   
1               0.001714           0.001746          0.086635   
2               0.002170           0.002211          0.079761   
3               0.001483           0.001511          0.710516   
4               0.001731           0.001764          0.110294   
...                  ...                ...               ...   
29157           0.000502           0.000512          0.139949   
29158           0.001181           0.001203          0.168410   
29159           0.000497           0.000506          0.094990   
29160           0.001012           0.001031          0.481397   
29161           0.001240           0.001263          0.420662   

       ind_cder_fin_ult1  ind_cno_fin_ult1  ind_ctju_fin_ult1  \
0               0.001882          0.043023           0.001730   
1               0.001882          0.021497           0.001736   
2               0.002442

In [15]:
submit.to_csv('submit_small.csv',index = 0)

In [18]:
print(submit.loc[0,:].to_list())

[0.001708397176116705, 0.0017402548110112548, 0.11568717658519745, 0.0018817170057445765, 0.043022822588682175, 0.0017299930332228541, 0.0017599196871742606, 0.007300227414816618, 0.01372937299311161, 0.0018888972699642181, 0.0019998266361653805, 0.03323611989617348, 0.02305479161441326, 0.0028352446388453245, 0.0018536350689828396, 0.00494238780811429, 0.001933414489030838, 0.010585333220660686, 0.10961246490478516, 0.004022097215056419, 0.0018040604190900922, 0.13724760711193085, 0.18749406933784485, 0.288930207490921]


In [26]:
test_data = pd.read_csv('test_data.csv')
submit_list = ['ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1',
                'ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1',
                'ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1',
                'ind_deco_fin_ult1','ind_deme_fin_ult1','ind_dela_fin_ult1',
                'ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1',
                'ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1',
                'ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1',
                'ind_nomina_ult1','ind_nom_pens_ult1','ind_recibo_ult1']
submit_all = pd.DataFrame(columns = ['ncodpers','added_products'])
submit_all['ncodpers'] = test_data['user_id']
print(submit_all)

       ncodpers added_products
0        657788            NaN
1        657794            NaN
2        657770            NaN
3        658056            NaN
4        657869            NaN
...         ...            ...
29157   1166847            NaN
29158   1166605            NaN
29159   1166829            NaN
29160   1166827            NaN
29161   1166781            NaN

[29162 rows x 2 columns]


In [27]:
#由于预测结果是全部产品的购买概率，每个用户选取概率最高的7个产品依次作为提交结果
for num_1 in tqdm(range(len(submit))):
    temp_list1 = submit.loc[num_1,:].to_list()
    temp_list1 = np.array(temp_list1)
    temp_1 = np.argsort(-temp_list1)
    submit_one = ''
    for num_2 in range(7):
        submit_one += submit_list[temp_1[num_2]] + ' '
    submit_all.loc[ num_1 , 'added_products'] = submit_one


 15%|███████████▎                                                                | 4333/29162 [00:06<00:40, 614.29it/s]


 28%|█████████████████████▌                                                      | 8296/29162 [00:13<00:39, 532.37it/s]


 41%|██████████████████████████████▌                                            | 11886/29162 [00:20<00:34, 499.59it/s]


 52%|███████████████████████████████████████▏                                   | 15228/29162 [00:27<00:32, 425.18it/s]


 62%|██████████████████████████████████████████████▎                            | 18006/29162 [00:34<00:35, 315.54it/s]


 72%|██████████████████████████████████████████████████████                     | 21027/29162 [00:42<00:22, 366.37it/s]


 82%|█████████████████████████████████████████████████████████████▋             | 23994/29162 [00:49<00:11, 440.59it/s]


 92%|█████████████████████████████████████████████████████████████████████▎     | 26928/29162 [00:56<00:05, 394.08it/s]


 99%|██████████████████████████████████████████████████████████████████████████▏| 28865/29162 [01:03<00:01, 225.83it/s]


100%|███████████████████████████████████████████████████████████████████████████| 29162/29162 [01:05<00:00, 448.30it/s]


In [28]:
print(submit_all)

       ncodpers                                     added_products
0        657788  ind_recibo_ult1 ind_nom_pens_ult1 ind_nomina_u...
1        657794  ind_recibo_ult1 ind_tjcr_fin_ult1 ind_nom_pens...
2        657770  ind_recibo_ult1 ind_nom_pens_ult1 ind_nomina_u...
3        658056  ind_cco_fin_ult1 ind_recibo_ult1 ind_nom_pens_...
4        657869  ind_recibo_ult1 ind_nomina_ult1 ind_nom_pens_u...
...         ...                                                ...
29157   1166847  ind_recibo_ult1 ind_cco_fin_ult1 ind_nom_pens_...
29158   1166605  ind_recibo_ult1 ind_cco_fin_ult1 ind_tjcr_fin_...
29159   1166829  ind_recibo_ult1 ind_cco_fin_ult1 ind_nom_pens_...
29160   1166827  ind_cco_fin_ult1 ind_recibo_ult1 ind_cno_fin_u...
29161   1166781  ind_recibo_ult1 ind_cco_fin_ult1 ind_tjcr_fin_...

[29162 rows x 2 columns]


In [29]:
submit_all.to_csv('sub_samll.csv',index=0)